In [ ]:
from io import StringIO

import folium

# import altair as alt
import geopandas as gpd
import pandas as pd
from folium.features import DivIcon
from folium.map import FeatureGroup, LayerControl
from folium.plugins import HeatMapWithTime
from shapely import make_valid

from utils.data import carrega_locale_altair, gpd_merge
from utils.geo import cria_mapa_com_graficos, json_municipios
# Configurações de locale para Altair para exibição em notebooks
# Descomentar para funcionar corretamente em notebooks
# alt.renderers.set_embed_options(format_locale="pt-BR", time_format_locale="pt-BR")


In [ ]:
# Lê o arquivo CSV removendo aspas duplas
# O CSV original é um CSV "sujo", criado com Excel, que adiciona aspas duplas
# em torno de cada campo, o que causa problemas na leitura.
with open(
    "../data/dados_exemplo_poluentes_no_acentos.csv", "r", encoding="utf-8-sig"
) as f:
    linhas = [linha.replace('"', "") for linha in f]

# Converte as linhas limpas para um buffer em memória
csv_buffer = StringIO("".join(linhas))

# Lê o CSV normalmente após a limpeza
df = pd.read_csv(csv_buffer, sep=",")
#
# Remove a coluna "unit" se todos os valores forem iguais
# Neste caso, todos os valores são "mg/L", então a coluna é desnecessária
if "unit" in df.columns and df["unit"].nunique() == 1:
    df = df.drop(columns=["unit"])

# Converte as colunas de latitude e longitude para o tipo numérico
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lon"] = pd.to_numeric(df["lon"], errors="coerce")

# Remove Estacao do nome das estações e converte para categoria
df["station_name"] = df["station_name"].str.replace("Estacao", "")
df["station_name"] = df["station_name"].str.strip()
df["station_name"] = df["station_name"].astype("category")

# Converte a coluna de data para o tipo datetime
df["sample_dt"] = pd.to_datetime(df["sample_dt"], format="%Y-%m-%d")

# Remove a coluna station_id, já que station_name é suficiente para identificar os pontos de coleta
# Cada station_id tem um station_name único
df = df.drop(columns=["station_id"])

# Transforma o DataFrame para o formato longo (long format) para facilitar a plotagem
# com bibliotecas de visualização
df_longo = df.melt(
    id_vars=["station_name", "lat", "lon", "sample_dt"],
    value_vars=["pol_a", "pol_b"],
    var_name="pollutant",
    value_name="value",
)

gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df["lon"], df["lat"]), crs="EPSG:4326"
)


In [ ]:
estados_sudeste = "ES", "MG", "RJ", "SP"

municipios_sudeste = json_municipios(estados_sudeste)
municipios_sudeste["geometry"] = make_valid(municipios_sudeste["geometry"])
pontos_coleta = (
    gdf[["station_name", "geometry"]].drop_duplicates().reset_index(drop=True)
)

pontos_coleta_municipios = gpd.sjoin(
    municipios_sudeste, pontos_coleta, how="inner", predicate="intersects"
).drop(columns="index_right")

pontos_coleta_municipios = gpd_merge(
    gdf,
    pontos_coleta_municipios[["station_name", "city", "state"]],
    on="station_name",
    how="left",
)

non_cat_cols = pontos_coleta_municipios.select_dtypes(exclude=["category"]).columns
pontos_coleta_municipios[non_cat_cols] = pontos_coleta_municipios[non_cat_cols].fillna(
    "N/A"
)


In [ ]:
pontos_coleta_municipios_longo = pontos_coleta_municipios.melt(
    id_vars=["station_name", "lat", "lon", "sample_dt", "city", "state", "geometry"],
    value_vars=["pol_a", "pol_b"],
    var_name="pollutant",
    value_name="value",
)

pontos_coleta_municipios_longo["sample_dt"] = pd.to_datetime(
    pontos_coleta_municipios_longo["sample_dt"], format="%d/%m/%Y"
)

pontos_coleta_municipios_longo = pontos_coleta_municipios_longo.sort_values("sample_dt")


In [ ]:
locale = carrega_locale_altair("pt-BR")
m = cria_mapa_com_graficos(pontos_coleta_municipios_longo, locale)
m.save("../maps/mapa.html")

In [ ]:
# Função para adicionar HeatMap
def adicionar_heatmap(mapa, df_longo):
    # Filtra os dados para o HeatMap
    dados_heatmap_pol_a = df_longo[df_longo["pollutant"] == "pol_a"]
    dados_heatmap_pol_b = df_longo[df_longo["pollutant"] == "pol_b"]

    # Agrupar por data
    #HACK: Trocar por uma operação vetorizada
    grupos_pol_a = dados_heatmap_pol_a.groupby("sample_dt").apply(
        lambda x: x[["lat", "lon", "value"]].values.tolist()
    )
    grupos_pol_b = dados_heatmap_pol_b.groupby("sample_dt").apply(
        lambda x: x[["lat", "lon", "value"]].values.tolist()
    )

    # Adiciona o HeatMap no mapa
    HeatMapWithTime(
        grupos_pol_a.tolist(),
        index=grupos_pol_a.index.tolist(),
        min_opacity=0.2,
        max_opacity=0.8,
        radius=15,
    ).add_to(mapa)
    HeatMapWithTime(
        grupos_pol_b.tolist(),
        index=grupos_pol_b.index.tolist(),
        min_opacity=0.2,
        max_opacity=0.8,
        radius=15,
    ).add_to(mapa)

    return mapa


m = folium.Map(
    location=[-23.5505, -46.6333], zoom_start=10
)  # Exemplo de localização (São Paulo)
m = adicionar_heatmap(m, pontos_coleta_municipios_longo)


In [ ]:
# Função para criar os ícones com mini-barras
def criar_icone_barras(pol_a, pol_b):
    # HTML para o DivIcon com duas barras
    html = f"""
    <div style="width: 30px; height: 40px; position: relative;">
        <div style="width: {pol_a}% ; height: 5px; background-color: red; position: absolute; top: 0;"></div>
        <div style="width: {pol_b}% ; height: 5px; background-color: blue; position: absolute; bottom: 0;"></div>
    </div>
    """
    return DivIcon(icon_size=(30, 40), html=html)


# Adicionando os ícones das estações
for idx, row in pontos_coleta_municipios_longo.iterrows():
    lat, lon = row["lat"], row["lon"]
    pol_a, pol_b = (
        row["value"] if row["pollutant"] == "pol_a" else None,
        row["value"] if row["pollutant"] == "pol_b" else None,
    )
    if pol_a is not None and pol_b is not None:
        icon = criar_icone_barras(pol_a, pol_b)
        folium.Marker([lat, lon], icon=icon).add_to(m)


In [ ]:
from folium.plugins import TimestampedGeoJson

# Adicionando o controle de camadas
folium.LayerControl().add_to(m)
# A função TimestampedGeoJson pode ser usada para adicionar o tempo aos pontos de coleta
#BUG: Não consegui fazer funcionar de jeito nenhum, o problema para ser a ordem
# que os plugins JS carregam

# Gerando um GeoJSON para ser usado no TimestampedGeoJson
geojson_data = pontos_coleta_municipios_longo[
    ["lat", "lon", "sample_dt", "station_name"]
].to_dict(orient="records")

timestamped_geojson = TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [d["lon"], d["lat"]]},
                "properties": {
                    "time": d["sample_dt"].strftime("%Y-%m-%d"),
                    "popup": f"Estação: {d['station_name']}<br>Data: {d['sample_dt']}",
                },
            }
            for d in geojson_data
        ],
    },
    period="PT1D",
    add_last_point=True,
)

timestamped_geojson.add_to(m)
m.save("mapa_final.html")

In [ ]:
pontos_coleta_municipios_longo = pd.read_parquet(
    "../data/pontos_coleta_municipios_longo.parquet"
)

In [ ]:
pontos = pontos_coleta_municipios_longo.copy()

pontos = pontos.dropna(subset=["lat", "lon", "value"])
pontos["sample_dt"] = pd.to_datetime(pontos["sample_dt"], errors="coerce")
pontos = pontos.dropna(subset=["sample_dt"])


In [ ]:
# Cria mapa base com tiles
m = folium.Map(location=[pontos["lat"].mean(), pontos["lon"].mean()], zoom_start=6)
m


In [ ]:
# ============ HEATMAP POL_A ============
pol_a_data = pontos[pontos["pollutant"] == "pol_a"]
heat_data_pol_a = (
    pol_a_data.groupby("sample_dt")
    .apply(
        #HACK Trocar por uma operação vetorizada
        lambda g: [[row["lat"], row["lon"], row["value"]] for _, row in g.iterrows()]
    )
    .tolist()
)
heat_dates_pol_a = sorted(pol_a_data["sample_dt"].dt.strftime("%Y-%m-%d").unique())

fg_pol_a = FeatureGroup(name="Heatmap pol_a")
HeatMapWithTime(
    heat_data_pol_a,
    index=heat_dates_pol_a,
    name="Heatmap pol_a",
    radius=15,
    auto_play=False,
    max_opacity=0.7,
).add_to(fg_pol_a)
fg_pol_a.add_to(m)
LayerControl(collapsed=False).add_to(m)
# m.save("_test_1.html")
m

In [ ]:
# ============ 🔥 HEATMAP POL_B ============
pol_b_data = pontos[pontos["pollutant"] == "pol_b"]
heat_data_pol_b = (
    pol_b_data.groupby("sample_dt")
    .apply(
        lambda g: [[row["lat"], row["lon"], row["value"]] for _, row in g.iterrows()]
    )
    .tolist()
)
heat_dates_pol_b = sorted(pol_b_data["sample_dt"].dt.strftime("%Y-%m-%d").unique())

fg_pol_b = FeatureGroup(name="Heatmap pol_b")
HeatMapWithTime(
    heat_data_pol_b,
    index=heat_dates_pol_b,
    name="Heatmap pol_b",
    radius=15,
    auto_play=False,
    max_opacity=0.7,
).add_to(fg_pol_b)
fg_pol_b.add_to(m)
m

In [ ]:
# Os elementos não renderizam no HTML
# Verificando a estrutura dos dados
print(len(heat_data_pol_b))
print(heat_data_pol_b[0][:3])
print(heat_dates_pol_b[:3])


In [ ]:
# ============ 📊 MINI-BARRAS ============
#Talvez criar uma função para criar os ícones com mini-barras
fg_barras = FeatureGroup(name="Mini-barras")

# Agrupar o último valor disponível por estação
# Serve como proxy, talvez seja melhor usar mediana ou média?
#TODO: Melhorar a lógica de agregação
latest = pontos.sort_values("sample_dt").drop_duplicates(
    ["station_name", "pollutant"], keep="last"
)
pivot = latest.pivot(
    index=["station_name", "lat", "lon"], columns="pollutant", values="value"
).reset_index()

for _, row in pivot.iterrows():
    pol_a_val = row.get("pol_a", 0)
    pol_b_val = row.get("pol_b", 0)

    # Normaliza de 0 a 100 para largura da barra
    pol_a_bar = min(
        max(int(pol_a_val * 10), 0), 100
    )  # Ajuste a escala conforme necessário
    pol_b_bar = min(max(int(pol_b_val * 10), 0), 100)

    icon_html = f"""
    <div style="width: 40px; height: 40px; border: 1px solid #888; background-color: white; text-align: center; font-size: 10px;">
        <div style="height: 8px; width: {pol_a_bar}%; background-color: red; margin: 2px auto;"></div>
        <div style="height: 8px; width: {pol_b_bar}%; background-color: blue; margin: 2px auto;"></div>
    </div>
    """

    folium.Marker(
        location=[row["lat"], row["lon"]],
        icon=DivIcon(html=icon_html),
        tooltip=folium.Tooltip(
            f"{row['station_name']}<br>pol_a: {pol_a_val:.2f}<br>pol_b: {pol_b_val:.2f}"
        ),
    ).add_to(fg_barras)

fg_barras.add_to(m)

# ============ ✅ LAYER CONTROL ============
LayerControl(collapsed=False).add_to(m)

# ============ 💾 SALVAR MAPA ============

print("✅ Mapa gerado com sucesso: mapa.html")
m